# Setup Enviorment

In [ ]:
%cd ..

In [1]:
from Preprocessing.selfsupervised_data_praparation import plot_samples, get_dataset_list
from Preprocessing.generate_embeddings import generate_embeddings_df, save_embeddings_as_csv
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNetV2, VGG16, ResNet50V2

import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd 

In [2]:
# Path to Dataset
path = '../Dataset_10_best_cities/'
model_name = 'ResNet50V2' #'VGG16' #'MobileNetV2' # 'vit' # 'autoencoder' #'variational_autoencoder'
latent_dim = 1024

target_size = (224, 224, 1)
model_input = (224, 224, 3)

# Model path
model_path = f'Weights/{model_name}_{target_size[0]}_{latent_dim}.h5'  

## Read dataset

In [3]:
#plot_samples(path)

In [4]:
image_list = get_dataset_list(path, show_dirs=True, head=0)

Images in directories: 
54001
41001
5001
50001
68001
8001
23001
76001
73001
5360


## Load Model

In [5]:
tf.random.set_seed(0)

if model_name == 'variational_autoencoder':
    from Models import Variational_Autoencoder
    model = Variational_Autoencoder.get_Variational_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim, lr=0.0001)
elif model_name == 'autoencoder':
    from Models import Autoencoder
    model = Autoencoder.get_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim)
elif model_name == 'vit':
    from Models import ViT
    model = ViT.get_vit_backbone(model_input)
elif model_name == 'MobileNetV2':
    cnn = MobileNetV2(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'VGG16': # min depth
    cnn = VGG16(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'ResNet50V2':
    cnn = ResNet50V2(input_shape=model_input, include_top=False, weights='imagenet') 

if model_name in ['MobileNetV2', 'VGG16', 'ResNet50V2']:
    model = Sequential()
    model.add(cnn)
    model.add(tf.keras.layers.GlobalAveragePooling2D())

    # freeze:
    for layer in model.layers:
        layer.trainable = False
        
model.summary()

2023-03-05 17:32:57.766461: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-05 17:32:57.766504: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-05 17:32:57.766528: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (9f7bdfccc75e): /proc/driver/nvidia/version does not exist
2023-03-05 17:32:57.766712: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
Total params: 23,564,800
Trainable params: 0
Non-trainable params: 23,564,800
_________________________________________________________________


## Generate embeddings

In [6]:
for band in range(12):
    print(f'Generating Embeddings for the band {band+1} with the model {model_name}...')
    # Embeddings path
    embeddings_path = f'Embeddings/{model_name}_Per_Band/{target_size[0]}_band{band+1}.csv'
    # Generate Embeddings
    embeddings = generate_embeddings_df(image_list=image_list, model=model, target_size=target_size, BAND=band)
    # Save Embeddings
    save_embeddings_as_csv(df=embeddings, path=embeddings_path)
    print('Done!')

Generating Embeddings for the band 1 with the model ResNet50V2...


2023-03-05 17:32:59.220433: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Done!
Generating Embeddings for the band 2 with the model ResNet50V2...
Done!
Generating Embeddings for the band 3 with the model ResNet50V2...
Done!
Generating Embeddings for the band 4 with the model ResNet50V2...
Done!
Generating Embeddings for the band 5 with the model ResNet50V2...
Done!
Generating Embeddings for the band 6 with the model ResNet50V2...
Done!
Generating Embeddings for the band 7 with the model ResNet50V2...
Done!
Generating Embeddings for the band 8 with the model ResNet50V2...
Done!
Generating Embeddings for the band 9 with the model ResNet50V2...
Done!
Generating Embeddings for the band 10 with the model ResNet50V2...
Done!
Generating Embeddings for the band 11 with the model ResNet50V2...
Done!
Generating Embeddings for the band 12 with the model ResNet50V2...
Done!


In [7]:
pd.read_csv(embeddings_path)

,Municipality Code,Date,0,1,2,3,4,5,6,7,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,54001,2017-07-30,0.0,0.062696,0.600844,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.317343,0.140887,0.0,0.725646,0.000000
1,54001,2015-12-20,0.0,0.000000,0.032145,0.000000,0.158393,0.0,0.000000,0.000000,...,0.000000,0.000000,0.124439,0.000000,0.0,2.590470,0.093306,0.0,1.749229,0.290249
2,54001,2017-11-19,0.0,0.000000,0.333641,0.000000,0.183420,0.0,0.000000,0.000000,...,0.000000,0.000000,0.096049,0.000000,0.0,1.642405,0.051209,0.0,1.816982,0.073031
3,54001,2016-11-20,0.0,1.122320,0.126140,0.062106,0.000000,0.0,0.075968,0.000000,...,0.059202,0.002387,0.063546,0.000000,0.0,0.260772,1.827995,0.0,1.373010,0.000865
4,54001,2017-07-16,0.0,0.062696,0.600844,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.317343,0.140887,0.0,0.725646,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,5360,2017-04-16,0.0,0.000000,0.056366,0.209204,0.031281,0.0,0.000000,0.000000,...,0.000000,0.000000,0.204470,0.000312,0.0,0.000000,0.210224,0.0,0.142911,0.000000
1636,5360,2018-12-02,0.0,0.039524,0.733919,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.293185,0.000000,0.0,2.659740,0.051901
1637,5360,2018-05-20,0.0,0.014366,0.000000,0.000000,0.078753,0.0,1.542432,0.000000,...,0.000000,0.000000,1.715615,0.001959,0.0,1.995143,0.208354,0.0,0.435300,0.000000
1638,5360,2016-09-18,0.0,0.043386,0.103800,0.000000,0.000000,0.0,0.035973,0.009716,...,0.000000,0.000000,0.742256,0.000000,0.0,1.989376,0.091636,0.0,2.116268,0.000000
